In [7]:
import numpy as np
import polars as pl

def american_option_pricing(S, K, r, sigma, T, N, option_type='call', method='fast'):
    # Time increment
    dt = T / N
    
    # Up and down factors
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    
    # Risk-neutral probability
    p = (np.exp(r * dt) - d) / (u - d)
    
    # Initialize option price tree
    option_tree = np.zeros((N+1, N+1))
    
    # Terminal values for call or put option
    for i in range(N+1):
        option_tree[i, N] = max(0, (u**i) * (d**(N-i)) * S - K) if option_type == 'call' else max(0, K - (u**i) * (d**(N-i)) * S)
    
    # Work backwards through the tree
    if method == 'fast':
        for j in range(N-1, -1, -1):
            option_tree[:j+1, j] = np.maximum(
                (u**np.arange(j+1)) * (d**(j-np.arange(j+1))) * S - K if option_type == 'call' else K - (u**np.arange(j+1)) * (d**(j-np.arange(j+1))) * S,
                np.exp(-r * dt) * (p * option_tree[1:j+2, j+1] + (1-p) * option_tree[:j+1, j+1])
            )
    else:  # method == 'slow'
        for j in range(N-1, -1, -1):
            for i in range(j+1):
                exercise_value = (u**i) * (d**(j-i)) * S - K if option_type == 'call' else K - (u**i) * (d**(j-i)) * S
                continuation_value = np.exp(-r * dt) * (p * option_tree[i+1, j+1] + (1-p) * option_tree[i, j+1])
                option_tree[i, j] = max(exercise_value, continuation_value)
    
    # Option price at the root of the tree
    option_price = option_tree[0, 0]
    
    # Greeks (approximations)
    delta = (option_tree[1, 1] - option_tree[0, 1]) / (u * S - S)
    gamma = ((option_tree[2, 2] - option_tree[1, 2]) / (u * u * S - u * S) - (option_tree[1, 2] - option_tree[0, 2]) / (u * S - S)) / ((u * S - S) / 2)
    theta = (option_tree[0, 1] - option_price) / dt
    
    # Note: Vega and Rho would require re-evaluating the tree with changes in sigma and r, respectively
    
    return option_price, delta, gamma, theta

# Example usage
S = 100  # Underlying asset price
K = 100  # Strike price
r = 0.05 # Risk-free interest rate
sigma = 0.2 # Volatility
T = 1    # Time to expiration (in years)
N = 100  # Number of time steps

call_price_fast, call_delta_fast, call_gamma_fast, call_theta_fast = american_option_pricing(S, K, r, sigma, T, N, option_type='call', method='fast')
put_price_fast, put_delta_fast, put_gamma_fast, put_theta_fast = american_option_pricing(S, K, r, sigma, T, N, option_type='put', method='fast')

call_price_slow, call_delta_slow, call_gamma_slow, call_theta_slow = american_option_pricing(S, K, r, sigma, T, N, option_type='call', method='slow')
put_price_slow, put_delta_slow, put_gamma_slow, put_theta_slow = american_option_pricing(S, K, r, sigma, T, N, option_type='put', method='slow')

# Create a Polars DataFrame
df = pl.DataFrame({
    "Option Type": ["Call", "Put", "Call", "Put"],
    "Method": ["Fast", "Fast", "Slow", "Slow"],
    "Price": [call_price_fast, put_price_fast, call_price_slow, put_price_slow],
    "Delta": [call_delta_fast, put_delta_fast, call_delta_slow, put_delta_slow],
    "Gamma": [call_gamma_fast, put_gamma_fast, call_gamma_slow, put_gamma_slow],
    "Theta": [call_theta_fast, put_theta_fast, call_theta_slow, put_theta_slow],
    "S": [S, S, S, S],
    "K": [K, K, K, K],
    "r": [r, r, r, r],
    "sigma": [sigma, sigma, sigma, sigma],
    "T": [T, T, T, T],
    "N": [N, N, N, N]
})

print(df)


shape: (4, 12)
┌─────────────┬────────┬───────────┬──────────┬───┬──────┬───────┬─────┬─────┐
│ Option Type ┆ Method ┆ Price     ┆ Delta    ┆ … ┆ r    ┆ sigma ┆ T   ┆ N   │
│ ---         ┆ ---    ┆ ---       ┆ ---      ┆   ┆ ---  ┆ ---   ┆ --- ┆ --- │
│ str         ┆ str    ┆ f64       ┆ f64      ┆   ┆ f64  ┆ f64   ┆ i64 ┆ i64 │
╞═════════════╪════════╪═══════════╪══════════╪═══╪══════╪═══════╪═════╪═════╡
│ Call        ┆ Fast   ┆ 10.430612 ┆ 1.26042  ┆ … ┆ 0.05 ┆ 0.2   ┆ 1   ┆ 100 │
│ Put         ┆ Fast   ┆ 6.082354  ┆ -0.81512 ┆ … ┆ 0.05 ┆ 0.2   ┆ 1   ┆ 100 │
│ Call        ┆ Slow   ┆ 10.430612 ┆ 1.26042  ┆ … ┆ 0.05 ┆ 0.2   ┆ 1   ┆ 100 │
│ Put         ┆ Slow   ┆ 6.082354  ┆ -0.81512 ┆ … ┆ 0.05 ┆ 0.2   ┆ 1   ┆ 100 │
└─────────────┴────────┴───────────┴──────────┴───┴──────┴───────┴─────┴─────┘
